In [1]:
import numpy as np
import pandas as pd
import pprint
from scapy.all import *
from collections import defaultdict
from IPy import IP as IPy
from manuf import manuf

In [2]:
# change this to the filename that you want to convert to a dataframe

filename = "./packet_capture.pcap"

In [3]:
packets = rdpcap(filename)

In [27]:
p_dict = {'Ethernet': ['dst', 'src', 'type'],
          'IP': ['version', 'ihl', 'tos', 'len', 'id', 'flags', 'frag', 'ttl', 'proto', 'chksum', 'src', 'dst'],
          'TCP': ['sport', 'dport', 'seq', 'ack', 'dataofs', 'reserved', 'flags', 'window', 'chksum', 'urgptr', 'options'],
          'Raw': ['load']       
         }

feature_dict = defaultdict(str)
feature_list = []

# Probably a better way to do this other than 3 nested loops!
# On the small file takes 5 secs to run but on a bigger file will probalby be -a lot- worse
# Goes through each packet and each of the key value pairs in the dict above to grab
# each value from the raw file to make into a dataframe

count = 0
for p in packets:
    for key in p_dict.keys():
        for cat in p_dict[key]:
            col_key = key + '_' + cat
            call = '"%' + key + '.' + cat + '%"'
            
            try:           
                feature_dict[col_key] = p.sprintf(call)
            except:
                feature_dict[col_key] = 'Null'
    
    feature_list.append(feature_dict)
    count += 1
    if count > 2:
        break

In [28]:
df = pd.DataFrame(feature_list)

In [29]:
df

,Ethernet_dst,Ethernet_src,Ethernet_type,IP_chksum,IP_dst,IP_flags,IP_frag,IP_id,IP_ihl,IP_len,...,TCP_chksum,TCP_dataofs,TCP_dport,TCP_flags,TCP_options,TCP_reserved,TCP_seq,TCP_sport,TCP_urgptr,TCP_window
0,"""??""","""??""","""??""","""0x56c""","""151.101.1.254""","""DF""","""0""","""55125""","""5""","""797""",...,"""0x213c""","""8""","""https""","""PA""","""[('NOP', None), ('NOP', None), ('Timestamp', ...","""0""","""3463306170""","""59554""","""0""","""9675"""
1,"""??""","""??""","""??""","""0x56c""","""151.101.1.254""","""DF""","""0""","""55125""","""5""","""797""",...,"""0x213c""","""8""","""https""","""PA""","""[('NOP', None), ('NOP', None), ('Timestamp', ...","""0""","""3463306170""","""59554""","""0""","""9675"""
2,"""??""","""??""","""??""","""0x56c""","""151.101.1.254""","""DF""","""0""","""55125""","""5""","""797""",...,"""0x213c""","""8""","""https""","""PA""","""[('NOP', None), ('NOP', None), ('Timestamp', ...","""0""","""3463306170""","""59554""","""0""","""9675"""
